In [1]:
sample_1 = "Staring at the contents of your fridge but never deciding what to eat is a cool way to diet"
print(sample_1)

Staring at the contents of your fridge but never deciding what to eat is a cool way to diet


In [8]:
import ollama
from pydantic import BaseModel, Field, ValidationError

class SarcasmClassifier(BaseModel):
    class_label: str = Field(
        description="""Label indicating whether the tweet is sarcastic or non_sarcastic.
            Possible values: 'sarcastic', 'non_sarcastic'."""
    )
    sarcasm_type: str = Field(
        description="""Type of speech present in the tweet if it is sarcastic.
            Possible values: 'sarcasm', 'irony', 'satire', 'understatement',
            'overstatement', 'rhetorical_question'. Should be null if class_label is 'non_sarcastic'."""
    )
    justification: str = Field(
        description="""Explain in brief, why the tweet is sarcastic or non-sarcastic.
        If sarcastic, explain why you chose this specific type of ironic speech."""
    )

In [9]:
system_prompt = """
You are an expert linguist specializing in analyzing ironic speech and sarcasm in social media communication.
Your task is to analyze tweets and:
1. Classify them as either sarcastic or non_sarcastic
2. For sarcastic tweets, identify the specific type of ironic speech being used
3. Provide a brief explanation for your classification results

Consider these definitions carefully when making your classifications:

Types of Ironic Speech:
- Sarcasm: Statements that contradict the state of affairs and contain explicit or implicit criticism of an addressee
- Irony: Statements that contradict the state of affairs but lack obvious criticism towards any particular addressee
- Satire: Statements that superficially appear supportive but contain underlying mockery or disagreement
- Understatement: Statements that deliberately minimize or underplay the significance of a situation
- Overstatement: Statements that use obvious exaggeration to describe a situation
- Rhetorical Question: Questions whose implied answer contradicts the actual state of affairs

Follow these steps for analysis:
1. First, determine if the tweet contains any form of ironic speech
2. If yes, label it as "sarcastic" and identify the specific type
3. If no, label it as "non_sarcastic"
4. Provide reasoning for your labelling.

Return only the classification label, sarcasm type (if applicable), and justification. Do Not provide any additional text.
"""
user_prompt = """
Please analyze the following tweet and classify it according to the system above:

Tweet to classify: {tweet_sample}
"""

In [10]:
def extract_info(tweet):
    response = ollama.chat(
        model="llama3.2",
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_prompt.format(tweet_sample = tweet)
            }
        ],
        options = {"temperature": 0.0},
        format = SarcasmClassifier.model_json_schema()
    )

    try:
        person_info = SarcasmClassifier.model_validate_json(response.message.content)
        return person_info
    except ValidationError as e:
        print(f"Error: {e}")
        return None

In [11]:
p2_res = extract_info(sample_1)

In [12]:
p2_res

SarcasmClassifier(class_label='sarcastic', sarcasm_type='irony', justification="The tweet states that staring at the fridge contents is a 'cool' way to diet, which contradicts the typical expectation of dieting being about making healthy choices. The tone implies criticism towards the idea of using this method.")